<a href="https://colab.research.google.com/github/Yasser023/D-fi-IA-Seryana/blob/main/GridSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
hotels = pd.read_csv('features_hotels.csv', index_col=['hotel_id', 'city'])
sample = pd.read_csv('sample_submission.csv')
test_set = pd.read_csv('test_set.csv')
df1 = pd.read_csv('dataset_1.csv')
df2 = pd.read_csv('dataset_2.csv')
df3 = pd.read_csv('dataset_3.csv')
df4 = pd.read_csv('dataset_4.csv')
df5 = pd.read_csv('dataset_5.csv')
df6 = pd.read_csv('dataset_6.csv')
df1000_1 = pd.read_csv('dataset_1000_1.csv')
df1000_2 = pd.read_csv('dataset_1000_2.csv')
df1000_3 = pd.read_csv('dataset_1000_3.csv')
dfv2_1 = pd.read_csv('dataset_v2_1.csv')
dfv2_2 = pd.read_csv('dataset_v2_2.csv')
#df_yanaser_1 = pd.read_csv('yanaser_1.csv')
#df_yanaser_2 = pd.read_csv('yanaser_2.csv')

df = pd.concat([df1,df2,df3,df4,df5,df6])
df = pd.concat([df1,df2,df3,df4,df5,df6,df1000_1,df1000_2,df1000_3,dfv2_1,dfv2_2])
               #,df_yanaser_1,df_yanaser_2])
df_final = df.join(hotels, on=['hotel_id', 'city'])
dataset = df_final.copy()
dataset = dataset.drop(['Unnamed: 0','hotel_id','avatar_id'], axis=1)
test_set_joined = test_set.join(hotels, on=['hotel_id', 'city'])
index = test_set_joined['index']
y = df_final['price']
X = df_final.drop(['Unnamed: 0','price'], axis=1)
X_test = test_set_joined[['hotel_id','stock','city','date','language','mobile', 'avatar_id','group','brand','parking','pool','children_policy']]
X_ids = X[['hotel_id','avatar_id']]
X_test_ids = X_test[['hotel_id','avatar_id']]
X = X.drop(['hotel_id','avatar_id'], axis=1)
X_test = X_test.drop(['hotel_id','avatar_id'], axis=1)

In [ ]:
pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from category_encoders import TargetEncoder
df_encoder = df_final.drop(['Unnamed: 0','hotel_id','avatar_id'], axis=1)
encoder1 = TargetEncoder()
encoder2 = TargetEncoder()
encoder3 = TargetEncoder()
encoder4 = TargetEncoder()
encoder1.fit(df_encoder['city'],df_encoder['price'])
encoder2.fit(df_encoder['language'],df_encoder['price'])
encoder3.fit(df_encoder['group'],df_encoder['price'])
encoder4.fit(df_encoder['brand'],df_encoder['price'])
df_encoder['city_encoded'] = encoder1.transform(df_encoder['city'])
df_encoder['language_encoded'] = encoder2.transform(df_encoder['language'])
df_encoder['group_encoded'] = encoder3.transform(df_encoder['group'])
df_encoder['brand_encoded'] = encoder4.transform(df_encoder['brand'])

XX_test = X_test.copy()
XX_test['city_encoded']=encoder1.transform(XX_test['city'])
XX_test['language_encoded']=encoder2.transform(XX_test['language'])
XX_test['group_encoded']=encoder3.transform(XX_test['group'])
XX_test['brand_encoded']=encoder4.transform(XX_test['brand'])
#XX_test.head()

/usr/local/lib/python3.8/dist-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/usr/local/lib/python3.8/dist-packages/category_encoders/target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [ ]:
#df_encoder.iloc[:, 0].values.reshape(-1,1)
#df_encoder.head()

In [ ]:
y = df_encoder.iloc[:, 0].values.reshape(-1,1)
df_train = df_encoder.drop(['price','city','language','group','brand'], axis=1)
XX_test = XX_test.drop(['city','language','group','brand'], axis=1)
print(np.shape(df_train))
print(np.shape(y))
print(np.shape(XX_test))

(252282, 10)
(252282, 1)
(6644, 10)


In [ ]:
# make a dictionary of hyperparameter values to search
search_space = {
    "n_estimators" : [100, 200, 500,1000],
    "max_depth" : [3, 6, 9],
    "gamma" : [0.01, 0.1],
    "learning_rate" : [0.001, 0.01, 0.1, 1]
}

In [ ]:
from xgboost import XGBRegressor
xgb_model = XGBRegressor(random_state = 2022)
from sklearn.model_selection import GridSearchCV
# make a GridSearchCV object
GS = GridSearchCV(estimator = xgb_model,
                  param_grid = search_space,
                  scoring = ["r2", "neg_root_mean_squared_error"], #sklearn.metrics.SCORERS.keys()
                  refit = "r2",
                  cv = 3,
                  verbose = 4)

In [ ]:
GS.fit(df_train,y)

Fitting 3 folds for each of 96 candidates, totalling 288 fits
[22:49:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV 1/3] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=100; neg_root_mean_squared_error: (test=-164.349) r2: (test=-2.918) total time=   9.4s
[22:49:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV 2/3] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=100; neg_root_mean_squared_error: (test=-179.045) r2: (test=-2.433) total time=   9.5s
[22:49:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV 3/3] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=100; neg_root_mean_squared_error: (test=-171.488) r2: (test=-2.498) total time=   9.5s
[22:50:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear i

In [ ]:
print(GS.best_estimator_) # to get the complete details of the best model

NameError: ignored

In [ ]:
from xgboost import XGBRegressor
xgb_model = XGBRegressor(random_state=
                         ,n_estimators = 1000
                         ,max_depth = 9
                         ,learning_rate = 1
                         ,gamma=0.01)
xgb_model.fit(df_train,y)
prediction = xgb_model.predict(XX_test)

[13:43:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
regressor = XGBRegressor(learning_rate=0.01,  
                      colsample_bytree = 1,
                      subsample = 1,
                      n_estimators=1000, 
                      reg_alpha = 0.3,
                      max_depth=10, 
                      gamma=0)
regressor.fit(df_train,y)
prediction = regressor.predict(XX_test)

[15:19:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
prediction = pd.Series(prediction)
submission_grid = pd.concat([index,prediction], axis=1)
submission_grid.columns = ['index','price']
submission_grid.head()

,index,price
0,0,107.569046
1,1,144.542984
2,2,70.866638
3,3,59.987415
4,4,207.172272


In [ ]:
submission_grid.to_csv('XGB_parametres_YAA_no_yanaser.csv',index=False)